# Test

In [1]:
import os
from pyomo.environ import *
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

## Importing DataSet

In [2]:
file_path = 'Test.xlsx'
# Read the Excel file into a dictionary of DataFrames, where keys are sheet names
Instance_data = pd.read_excel(file_path, sheet_name=None)

In [3]:
# Exam Data Set

Exams = Instance_data['Sheet1']
Exams['Exams'] = Exams['Exams'] - 1
#Exams['Exams'] = Exams['Exams'] 
ExamsList = list(Exams['Exams'])

In [4]:
#ExamsList=set (ExamsList)
ExamsList

[0, 1, 2, 3]

### Dictionary of Students and their related Exams

In [5]:
# Creating a Dictionary of Students and Their Related Exams
Students_exams = Instance_data['Sheet3']
Students_exams['ExamNumber'] = Students_exams['ExamNumber'] - 1
# Initialize an empty dictionary to store the students and their exams
students = {}
# Iterate over the rows in the DataFrame
for index, row in Students_exams.iterrows():
    student = row['Student']
    exam_number = row['ExamNumber']
    
    # Check if the student is already in the dictionary
    if student in students:
        students[student].append(exam_number)
    # If the student is not in the dictionary, create a new key-value pair    
    else:
        students[student] = [exam_number]

In [6]:
students

{'s1 ': [0, 1, 2],
 's2 ': [0, 2],
 's3 ': [3],
 's4 ': [2],
 's5 ': [0, 2],
 's6 ': [3],
 's7 ': [1, 2],
 's8 ': [0, 1]}

### TimeSlots

In [7]:
Time_s = Instance_data['Sheet2']
time_slots = Time_s.loc[0, 'TimeSlot']
TimeList= list(range (time_slots))

In [8]:
TimeList

[0, 1, 2, 3, 4, 5]

### Distance

In [9]:
#Distance
Distance=list(range(1,6))
Distance

[1, 2, 3, 4, 5]

###  Finding Conflicted Exams (Weighted-Unweighted)

In [10]:
# Unweighted

# Step 1: Initialize the dictionary to store conflicting exams
conflicting_exams = {exam: set() for exams_list in students.values() for exam in exams_list}

# Step 2: Populate the dictionary with conflicting exams
for student_exams in students.values():
    for i, exam1 in enumerate(student_exams):
        for exam2 in student_exams[i + 1:]:
            conflicting_exams[exam1].add(exam2)
            conflicting_exams[exam2].add(exam1)

# Step 3: Create the unweighted conflicting matrix
Unweighted_Conflict_Matrix = [
    [1 if exam2 in conflicting_exams[exam1] else 0 for exam2 in ExamsList] for exam1 in ExamsList
]



In [11]:
# Weighted

# The weighted Conflicting MATRIX
# Step 1: Initialize the dictionary to store conflicting exams
conflicting_exams = {exam: set() for exams_list in students.values() for exam in exams_list}

# Step 2: Populate the dictionary with conflicting exams
for student_exams in students.values():
    for i, exam1 in enumerate(student_exams):
        for exam2 in student_exams[i + 1:]:
            conflicting_exams[exam1].add(exam2)
            conflicting_exams[exam2].add(exam1)

# Step 3: Create the weighted conflicting matrix
weighted_conflicting_matrix = [[0 for _ in ExamsList] for _ in ExamsList]

# Step 4: Calculate the number of students in each conflicting exam pair and fill the matrix
for i, exam1 in enumerate(ExamsList):
    for j, exam2 in enumerate(ExamsList):
        if exam2 in conflicting_exams[exam1]:
            count = sum(1 for student_exams in students.values() if exam1 in student_exams and exam2 in student_exams)
            weighted_conflicting_matrix[i][j] = count



### Converting the Llist of UC and WC to Matrix of Coefficients

In [13]:
#Conflicting Exams (Weighted and Unweighted)
UC_Matrix=np.array(Unweighted_Conflict_Matrix )
WC_Matrix=np.array(weighted_conflicting_matrix)

In [14]:
UC_Matrix

array([[0, 1, 1, 0],
       [1, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 0, 0, 0]])

In [15]:
WC_Matrix

array([[0, 2, 3, 0],
       [2, 0, 2, 0],
       [3, 2, 0, 0],
       [0, 0, 0, 0]])

In [16]:
# Setting up the environment
env = gp.Env(
    r"C:\Users\ghadi\Downloads\Discrete Optimization Project 2023\Discrete Optimization Project 2023-Log File.log")
env.start()

# Environment parameters
env.setParam("Threads", 1)  # Controls the number of threads to apply to parallel algorithms
env.setParam("Presolve", 1)  # Controls the presolve level. conservative (1).
env.setParam("MIPGap", 1e-4)
env.setParam('Method', 0)  # Algorithm used to solve the initial root relaxation of a MIP model. 0=primal simplex.
env.setParam("TimeLimit", 600)  # 10 minutes time limit
env.setParam("PreSparsify", 1) # to reduce the memory used


Set parameter Username
Set parameter LogFile to value "C:\Users\ghadi\Downloads\Discrete Optimization Project 2023\Discrete Optimization Project 2023-Log File.log"
Academic license - for non-commercial use only - expires 2024-07-07
Set parameter Threads to value 1
Set parameter Presolve to value 1
Set parameter Method to value 0
Set parameter TimeLimit to value 600
Set parameter PreSparsify to value 1


### Defining Gurobi Model

In [17]:
model = gp.Model("Exam_Scheduling",env=env)

### Defining Decision Variable and Auxiliary Variable

In [18]:
#Decision Variable
X = model.addVars(ExamsList, TimeList , vtype=GRB.BINARY, name="X")


In [19]:
# Binary auxiliary variable for controlling conflict
Y = model.addVars(ExamsList, ExamsList, Distance, vtype=GRB.BINARY, name="Y")

###  Defining Constraints 

In [20]:
#Constraint 1
model.addConstrs(UC_Matrix[i, j] * (X[i, k] + X[j, k]) <= 1 for i in ExamsList for j in ExamsList for k in TimeList)
model.update()

In [21]:
#Constraint 2
model.addConstrs(sum(X[i, k] for k in TimeList) == 1 for i in ExamsList for k in TimeList)
model.update()

In [22]:
#Constraint C3
model.addConstrs((X[i, k] + X[j, k + d]) <= Y[i, j, d] + 1 for i in ExamsList for j in ExamsList for k in TimeList for d in Distance if (k + d) in TimeList and UC_Matrix[i, j] != 0 )
model.update()

### Defining Objective Function

In [23]:
# Create the objective expression using a loop
objective_expr = 0
for i in ExamsList:
    for j in ExamsList:
        for d in Distance:
            objective_expr += Y[i, j, d] * WC_Matrix[i, j] * ((2 ** (5 - d))/8)

# Set up the objective function for minimization
model.setObjective(objective_expr, GRB.MINIMIZE)

In [24]:
model.update()

In [25]:
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 1 threads

Optimize a model with 210 rows, 104 columns and 486 nonzeros
Model fingerprint: 0x8198af83
Variable types: 0 continuous, 104 integer (104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 9.0000000
Presolve removed 111 rows and 56 columns
Presolve time: 0.00s
Presolved: 99 rows, 48 columns, 306 nonzeros
Variable types: 0 continuous, 48 integer (48 binary)

Root relaxation: objective 0.000000e+00, 42 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   

In [26]:
if model.Status == GRB.OPTIMAL:
    print("Optimization was successful and an optimal solution was found.")
elif model.Status == GRB.INFEASIBLE:
    print("The model is infeasible - no feasible solution exists.")
elif model.Status == GRB.UNBOUNDED:
    print("The model is unbounded - the objective can be improved without limit.")
print("Objective Value:", model.objVal)

Optimization was successful and an optimal solution was found.
Objective Value: 3.375


In [27]:
print("Objective Value:", model.objVal)

Objective Value: 3.375


### Time Slots and Exams in Each Time Slot

In [28]:
# Scheduled-Exams in LIST
scheduled_exams = []
# Print the values of decision variable X
for exam in ExamsList:
    for time in TimeList:
        if X[exam, time].x > 0.5:
            scheduled_exams.append((exam, time))

In [29]:
# Create a dictionary to store exams for each time slot
Exams_in_TimeSlots = {}

for exam, time in scheduled_exams:
    if time not in Exams_in_TimeSlots:
        Exams_in_TimeSlots[time] = []
    Exams_in_TimeSlots[time].append(exam)

# Print the organized structure
for time_slot, exams in sorted(Exams_in_TimeSlots.items()):
    print(f"Time Slot {time_slot}: Exams {exams}")

Time Slot 0: Exams [2]
Time Slot 3: Exams [1]
Time Slot 5: Exams [0, 3]


In [30]:
# Sorting the Dic According to Time Slot It Will be Sorted By Keys 
Exams_in_TimeSlots= dict(sorted(Exams_in_TimeSlots.items()))

In [31]:
Exams_in_TimeSlots

{0: [2], 3: [1], 5: [0, 3]}

### Chcking whether there is Conflict Between the Exams In Each Time Slots

In [32]:
#All pairs of Time Tables exams related to Each time slot-More General
All_pairs_in_TimeSlots_Exams = []

for time_slot, exams in Exams_in_TimeSlots.items():
    exam_pairs = [(exams[i], exams[j]) for i in range(len(exams)) for j in range(i + 1, len(exams))]
    All_pairs_in_TimeSlots_Exams.extend(exam_pairs)

In [33]:
#Conflicting Exams - The code is Correct
List_ConflictExams=[]
for i in ExamsList:
    for j in ExamsList:
        if UC_Matrix[i,j] > 0:
            List_ConflictExams.append((i,j))

In [34]:
# To find Whether There is Intersection Between This Lists or Not

Set_All_pairs_in_TimeSlots_Exams=set(All_pairs_in_TimeSlots_Exams)
Set_List_ConflictExams=set(List_ConflictExams)
Common_elements = Set_All_pairs_in_TimeSlots_Exams.intersection(List_ConflictExams)
# Find common elements
if len (Common_elements) == 0:
    print('The Solution is Feasible Meaning that There is No Conflicting Exam in a All Time Slots')
else:
    print('The Solution is Not Feasible Meaning that There is Conflicting Exam in Time Slots')
    


# Print the common elements
#print("Common Elements:", Common_elements)

The Solution is Feasible Meaning that There is No Conflicting Exam in a All Time Slots


### Necessary OutPuts 

In [35]:
output_folder = r"C:\Users\ghadi\Downloads\Discrete Optimization Project 2023"
output_file_path = os.path.join(output_folder, "Discrete Optimization Project 2023.sol")

# Specify the file name only for model.write()
model.write("Discrete Optimization Project 2023.sol")

with open(output_file_path, 'w') as f:
    # your code here
    
    # Convert Exams_in_TimeSlots to a string before writing
    exams_string = ", ".join(map(str, TimeList))
    f.write(f"Exams Time Slots are: {exams_string}\n")
    
    f.write('\n\n')
    
    if len (Common_elements) == 0:
        f.write('The Solution is Feasible Meaning that There is No Conflicting Exam in  All Time Slots')
    else:
        f.write('The Solution is Not Feasible Meaning that There is Conflicting Exam in Time Slots')
        
    
    f.write('\n\n')    
        
    
    for time_slot, exams in sorted(Exams_in_TimeSlots.items()):
        f.write(f"Time Slot {time_slot}: Exams {exams}\n")
        
    f.write('\n\n') 
    
    f.write(f"The Objected Value - Penalty: {model.objVal}\n")
    f.write(f"The Benchmark is = 3.375 \n")
    

### Advanced Model

In [163]:
# Finding Time Lists Without More than Three Consecutive Items
def has_consecutive_numbers(sublist):
    consecutive_count = 0
    for i in range(len(sublist) - 1):
        if sublist[i] + 1 == sublist[i + 1]:
            consecutive_count += 1
            if consecutive_count >= 3:
                return True
        else:
            consecutive_count = 0
    return False

def detect_and_remove_consecutive_sublists(input_list):
    all_sublists = unique_sublists(input_list)
    consecutive_sublists = []

    for sublist in all_sublists:
        if has_consecutive_numbers(sublist):
            consecutive_sublists.append(sublist)

    # Remove detected consecutive sublists
    for sublist in consecutive_sublists:
        all_sublists.remove(sublist)

    #print("Original List:", input_list)
    #print("All Sublists Without 3 Consecutive Numbers:", all_sublists)
    #print("\nSublists with more than three consecutive numbers:")
    #for sublist in consecutive_sublists:
    #    print(sublist)
    return all_sublists



In [166]:
# Example usage with the list [1, 2, 3, 4, 5]
#Lists Without More than 3 Consecutive Items
input_list = [1, 2, 3, 4, 5]
Prefered_List=detect_and_remove_consecutive_sublists(input_list)
Prefered_List

[[2],
 [3, 4],
 [5],
 [1, 2, 5],
 [2, 5],
 [1, 3],
 [1, 4, 5],
 [2, 4, 5],
 [4],
 [1],
 [4, 5],
 [1, 2, 4],
 [1, 2, 4, 5],
 [2, 4],
 [1, 2],
 [1, 3, 4, 5],
 [1, 5],
 [1, 3, 5],
 [2, 3, 5],
 [3],
 [3, 5],
 [1, 2, 3, 5],
 [1, 2, 3],
 [1, 4],
 [1, 3, 4],
 [2, 3],
 [3, 4, 5],
 [2, 3, 4],
 []]

In [155]:
# To Choose the Max Items of Values in the Dictionary 


student_dict = {
    's1': [0, 1, 2],
    's2': [0, 2],
    's3': [3],
    's4': [2],
    's5': [0, 2],
    's6': [3],
    's7': [1, 2],
    's8': [0, 1]
}

max_key = None
max_value = None
max_item_count = 0

for key, value in student_dict.items():
    item_count = len(value)
    #print(f"Key: {key}, Value: {value}, Number of Items: {item_count}")

    if item_count > max_item_count:
        max_key = key
        max_value = value
        max_item_count = item_count

print(f"\nKey and Value with the Maximum Number of Items:")
print(f"Key: {max_key}, Value: {max_value}, Number of Items: {max_item_count}")



Key and Value with the Maximum Number of Items:
Key: s1, Value: [0, 1, 2], Number of Items: 3


In [170]:
Filtered_Subsets_Times=[sublist for sublist in Prefered_List if len(sublist) >=3 ]

In [171]:
Filtered_Subsets_Times

[[1, 2, 5],
 [1, 4, 5],
 [2, 4, 5],
 [1, 2, 4],
 [1, 2, 4, 5],
 [1, 3, 4, 5],
 [1, 3, 5],
 [2, 3, 5],
 [1, 2, 3, 5],
 [1, 2, 3],
 [1, 3, 4],
 [3, 4, 5],
 [2, 3, 4]]

In [230]:
# Iterate through each sublist
for index, sublist in enumerate(Filtered_Subsets_Times):
    for item in sublist:
        print(item)
    #print(f"Sublist {index + 1}: {sublist}")

    # Perform operations on the sublist if needed
    # Example: Print the sum of the sublist
    #print(f"Sum of Sublist {index + 1}: {sum(sublist)}")
    #print("-" * 20)
    
    
    
    #print (index)


1
2
5
1
4
5
2
4
5
1
2
4
1
2
4
5
1
3
4
5
1
3
5
2
3
5
1
2
3
5
1
2
3
1
3
4
3
4
5
2
3
4


In [249]:
from gurobipy import Model, GRB

# Example data
ExamsList = [1, 2, 3, 4, 5]
TimeList = [1, 2, 3, 4]

# Create a Gurobi model
model = Model()

# Create decision variables for each pair of exam and time slot
X = model.addVars(((i, k) for i in ExamsList for k in TimeList), vtype=GRB.BINARY, name="X")

# At most three consecutive time slots can have conflicting exams
for i in ExamsList:
    for j in ExamsList:
        for k in range(len(TimeList) - 3):  # Iterate up to the third-to-last time slot
            # Conflict constraint for each set of four consecutive time slots
            model.addConstr(X[i, TimeList[k]] + X[j, TimeList[k + 1]] + X[i, TimeList[k + 2]] + X[j, TimeList[k + 3]] <= 3)

# Continue with the rest of your code...


In [250]:
from gurobipy import Model, GRB

# Example data
ExamsList = [1, 2, 3, 4, 5]
TimeList = [1, 2, 3, 4]

# Create a Gurobi model
X = model.addVars(((i, k) for i in ExamsList for k in TimeList), vtype=GRB.BINARY, name="X")

model = Model()


In [256]:
for i in ExamsList:
    for k in range(len(TimeList) - 3):  # Iterate up to the third-to-last time slot
        # Conflict constraint for each set of four consecutive time slots
        model.addConstr(quicksum(X[i, t] for t in TimeList[k:k+4]) + quicksum(X[j, t] for j in ExamsList for t in TimeList[k:k+4]) <= 3)




In [255]:
X

{(1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 4): <gurobi.Var *Awaiting Model Update*>,
 (2, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 4): <gurobi.Var *Awaiting Model Update*>,
 (3, 1): <gurobi.Var *Awaiting Model Update*>,
 (3, 2): <gurobi.Var *Awaiting Model Update*>,
 (3, 3): <gurobi.Var *Awaiting Model Update*>,
 (3, 4): <gurobi.Var *Awaiting Model Update*>,
 (4, 1): <gurobi.Var *Awaiting Model Update*>,
 (4, 2): <gurobi.Var *Awaiting Model Update*>,
 (4, 3): <gurobi.Var *Awaiting Model Update*>,
 (4, 4): <gurobi.Var *Awaiting Model Update*>,
 (5, 1): <gurobi.Var *Awaiting Model Update*>,
 (5, 2): <gurobi.Var *Awaiting Model Update*>,
 (5, 3): <gurobi.Var *Awaiting Model Update*>,
 (5, 4): <gurobi.Var *Awaiting Model Update*>}